# Introduction

Health care is moving towards a value-based care model that incentivizes health care providers for providing quality care, quantified by improvements of the patient's health outcomes. This model involves understanding the health care needs of patients, integrating a multidisciplinary team to develop solutions that impact population health, and measuring health outcomes of patients to drive ongoing quality improvement projects.

With the advent of electronic health data and recent innovations in the field of machine learning, large and complex clinical data have been used by health care professionals to have answer pressing problems. Machine learning methods have been used to improve patient risk stratification for specific infections, disease prediction, and streamlining hospital operations.

Centers for Medicare & Medicaid Services (CMS) have created various value-based programs that incentivizes health care providers to provide better care and value to patients. For instance, the [Hospital Readmission Reduction Program](https://www.cms.gov/index.php/Medicare/Quality-Initiatives-Patient-Assessment-Instruments/Value-Based-Programs/HRRP/Hospital-Readmission-Reduction-Program) focuses on improving care coordination efforts in order to reduce readmission rates, effectively saving health care costs for both the provider and the patient. 

The program defines readmission as:

1. Unplanned readmissions that happen within 30 days of discharge from the initial admission and;
2. Patients who are readmitted to the same hospital, or another applicable acute care hospital for any reason.

This notebook will focus on applying classification models to identify patients who are at risk for readmission, as defined by the HRRP. The data used for this project can be found in the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/diabetes+130-us+hospitals+for+years+1999-2008).

## Data Set Information

The data has been prepared to analyze factors related to readmission as well as other outcomes pertaining to patients with diabetes. This data has been used to study the impact of HbA1c measurement and hospital readmission rates (see [here](https://www.hindawi.com/journals/bmri/2014/781670/)).

As mentioned in the article, the data represents 10 years of clinical care at 130 US hospitals and integrated delivery networks. The data has been de-identified and each observations in the dataset satisfy the following criteria:

1. an inpatient encounter (i.e., hospital admission),
2. a diabetic encounter (i.e., any kind of diabetes has been identified as one the patient's diagnosis),
3. the length of stay was at least 1 day and at most 14 days,
4. laboratory tests were performed during the encounter and;
5. medications were administered during the encounter.

More information about the data can be found in the article in the following [link](https://www.hindawi.com/journals/bmri/2014/781670/).

# Methodology

## Data Exploration

In [1]:
# Installing packages.
import pandas as pd

In [2]:
# Load data into a pandas dataframe.
diabetes_df = pd.read_csv("diabetic_data.csv", low_memory=False)

# Returning number of observations and features of the data.
print(f'The dataset has {diabetes_df.shape[0]} observations and {diabetes_df.shape[1]} features.')

The dataset has 101766 observations and 50 features.


In [4]:
# Returning data types for the features.
diabetes_df.dtypes.value_counts()

object    37
int64     13
dtype: int64

In [13]:
# Returning categorical features.
diabetes_df.columns[diabetes_df.dtypes=='object']

Index(['race', 'gender', 'age', 'weight', 'payer_code', 'medical_specialty',
       'diag_1', 'diag_2', 'diag_3', 'max_glu_serum', 'A1Cresult', 'metformin',
       'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride',
       'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted'],
      dtype='object')

In [6]:
# Returning numerical features.
diabetes_df.columns[diabetes_df.dtypes!='object']

Index(['encounter_id', 'patient_nbr', 'admission_type_id',
       'discharge_disposition_id', 'admission_source_id', 'time_in_hospital',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient',
       'number_diagnoses'],
      dtype='object')

In [61]:
# Assigning categorical features into a variable.
categorical_cols = diabetes_df.columns[diabetes_df.dtypes=='object']

# Assisgnin an array of unique values for each categorical features.
unique_cat_cols = [diabetes_df[i].unique() for i in categorical_cols]

# Creating a dataframe to view results.
pd.set_option('display.max_colwidth', None)
pd.DataFrame({'Unique Values': unique_cat_cols}, index=categorical_cols)

,Unique Values
race,"[Caucasian, AfricanAmerican, ?, Other, Asian, Hispanic]"
gender,"[Female, Male, Unknown/Invalid]"
age,"[[0-10), [10-20), [20-30), [30-40), [40-50), [50-60), [60-70), [70-80), [80-90), [90-100)]"
weight,"[?, [75-100), [50-75), [0-25), [100-125), [25-50), [125-150), [175-200), [150-175), >200]"
payer_code,"[?, MC, MD, HM, UN, BC, SP, CP, SI, DM, CM, CH, PO, WC, OT, OG, MP, FR]"
medical_specialty,"[Pediatrics-Endocrinology, ?, InternalMedicine, Family/GeneralPractice, Cardiology, Surgery-General, Orthopedics, Gastroenterology, Surgery-Cardiovascular/Thoracic, Nephrology, Orthopedics-Reconstructive, Psychiatry, Emergency/Trauma, Pulmonology, Surgery-Neuro, Obsterics&Gynecology-GynecologicOnco, ObstetricsandGynecology, Pediatrics, Hematology/Oncology, Otolaryngology, Surgery-Colon&Rectal, Pediatrics-CriticalCare, Endocrinology, Urology, Psychiatry-Child/Adolescent, Pediatrics-Pulmonology, Neurology, Anesthesiology-Pediatric, Radiology, Pediatrics-Hematology-Oncology, Psychology, Podiatry, Gynecology, Oncology, Pediatrics-Neurology, Surgery-Plastic, Surgery-Thoracic, Surgery-PlasticwithinHeadandNeck, Ophthalmology, Surgery-Pediatric, Pediatrics-EmergencyMedicine, PhysicalMedicineandRehabilitation, InfectiousDiseases, Anesthesiology, Rheumatology, AllergyandImmunology, Surgery-Maxillofacial, Pediatrics-InfectiousDiseases, Pediatrics-AllergyandImmunology, Dentistry, Surgeon, Surgery-Vascular, Osteopath, Psychiatry-Addictive, Surgery-Cardiovascular, PhysicianNotFound, Hematology, Proctology, Obstetrics, SurgicalSpecialty, Radiologist, Pathology, Dermatology, SportsMedicine, Speech, Hospitalist, OutreachServices, Cardiology-Pediatric, Perinatology, Neurophysiology, Endocrinology-Metabolism, DCPTEAM, Resident]"
diag_1,"[250.83, 276, 648, 8, 197, 414, 428, 398, 434, 250.7, 157, 518, 999, 410, 682, 402, 737, 572, V57, 189, 786, 427, 996, 277, 584, 462, 473, 411, 174, 486, 998, 511, 432, 626, 295, 196, 250.6, 618, 182, 845, 423, 808, 250.4, 722, 403, 250.11, 784, 707, 440, 151, 715, 997, 198, 564, 812, 38, 590, 556, 578, 250.32, 433, V58, 569, 185, 536, 255, 250.13, 599, 558, 574, 491, 560, 244, 250.03, 577, 730, 188, 824, 250.8, 332, 562, 291, 296, 510, 401, 263, 438, 70, 250.02, 493, 642, 625, 571, 738, 593, 250.42, 807, 456, 446, 575, ...]"
diag_2,"[?, 250.01, 250, 250.43, 157, 411, 492, 427, 198, 403, 288, 998, 507, 174, 425, 456, 401, 715, 496, 428, 585, 250.02, 410, 999, 996, 135, 244, 41, 571, 276, 997, 599, 424, 491, 553, 707, 286, 440, 493, 242, 70, V45, 250.03, 357, 511, 196, 396, 197, 414, 250.52, 577, 535, 413, 285, 53, 780, 518, 150, 566, 250.6, 867, 486, V15, 8, 788, 340, 574, 581, 228, 530, 250.82, 786, 294, 567, 785, 512, 305, 729, 250.51, 280, 648, 560, 618, 444, 38, V10, 578, 277, 781, 250.42, 278, 426, 584, 462, 402, 153, 272, 733, 34, 881, ...]"
diag_3,"[?, 255, V27, 403, 250, V45, 38, 486, 996, 197, 250.6, 427, 627, 414, 416, 714, 428, 582, V43, 250.01, 263, 250.42, 276, 482, 401, 250.41, 585, 781, 278, 998, 568, 682, 618, 250.02, 305, 707, 496, 599, 715, 424, 518, 553, 794, 411, V42, 531, 511, 490, 562, 250.8, 250.7, 250.52, 784, 491, 581, 420, 8, 724, 730, 789, 131, 250.82, 999, 41, 493, 250.03, 753, 786, 529, E888, 425, 595, 303, 560, 711, 492, 332, 296, 438, 362, 250.4, 654, 244, V70, 737, 625, 681, 250.51, 404, V10, 810, 280, 440, 785, 588, 569, 272, 997, 250.43, 918, ...]"
max_glu_serum,"[None, >300, Norm, >200]"
